# Welcome to TQ42

## Example of Hybrid Quantum Long Short-Term Memory (HQLSTM)

In [12]:
from tq42.client import TQ42Client
from tq42.organization import list_all as list_all_organizations
from tq42.project import list_all as list_all_projects
from tq42.experiment import list_all as list_all_experiments
from tq42.experiment_run import ExperimentRun
from tq42.compute import HardwareProto
from tq42.algorithm import (
    AlgorithmProto, 
    OptimProto, 
    LossFuncProto, 
    DatasetStorageInfoProto, 
    TSHQLSTMTrainMetadataProto, 
    TSHQLSTMTrainParametersProto,
    MLTrainInputsProto
)
from google.protobuf.json_format import MessageToDict

# Create a client and selecting org, proj and exp

In [ ]:
with TQ42Client() as client:
    org_sample = list_all_organizations(client=client)[0] 
    proj_sample = list_all_projects(client=client, organization_id=org_sample.id)[0]
    exp_sample = list_all_experiments(client=client, project_id=proj_sample.id)[-1]

In [13]:
    print(f"Running experiment within: Org {org_sample.id}, Proj {proj_sample.id} and Exp {exp_sample.id}`")

# Set parameters for the train

In [14]:
parameters_train = MessageToDict(TSHQLSTMTrainMetadataProto(
    parameters=TSHQLSTMTrainParametersProto(
        input_width=1,
        label_width=1,
        hidden_size=17,
        num_qubits=8,
        depth=7,
        n_qlayers=3,
        dropout_coef=0.24,
        num_epochs=5,
        batch_size=20,
        learning_rate=0.05,
        optim=OptimProto.ADAM,
        loss_func=LossFuncProto.MSE,
        time_column="Time",
        target_column="Power, kW"
    ),
    inputs=MLTrainInputsProto(
        data=DatasetStorageInfoProto(storage_id="fc4310c5-a2a3-43cd-990c-8dd2b1aec994")
    )
), preserving_proto_field_name=True)

# Train the model

In [15]:
run = ExperimentRun.create(
    client=client,
    algorithm=AlgorithmProto.TS_HQLSTM_TRAIN, 
    experiment_id=exp_sample.id,
    compute=HardwareProto.SMALL, 
    parameters=parameters_train
)

print(run.data)

id: "640ec1a4-2f1f-4a1f-bec7-fbeaa5c0d06d"
experiment_id: "811fd59b-2b51-42ea-b573-b5aebb0d5d7a"
sequential_id: 26
status: QUEUED
algorithm: TS_HQLSTM_TRAIN
hardware: SMALL
metadata: "{\"parameters\":{\"input_width\":1,\"label_width\":1,\"hidden_size\":17,\"num_qubits\":8,\"depth\":7,\"dropout_coef\":0.24,\"num_epochs\":5,\"batch_size\":20,\"learning_rate\":0.05,\"optim\":\"ADAM\",\"loss_func\":\"MSE\",\"train_model_info\":null,\"nqlayers\":3},\"inputs\":{\"data\":{\"storage_id\":\"fc4310c5-a2a3-43cd-990c-8dd2b1aec994\"}}}"
result {
}
created_by: "35dc8cba-b06d-4f15-b9bc-588f608789de"
created_at {
  seconds: 1705664553
  nanos: 50279143
}


# Import functions for the evaluate

In [16]:
from tq42.algorithm import (
    TSHQLSTMEvalMetadataProto,
    TSHQLSTMEvalParametersProto,
    TSEvalInputsProto,
    ModelStorageInfoProto
)

# Set parameters for the evaluate

In [17]:
params_eval = MessageToDict(TSHQLSTMEvalMetadataProto(
    parameters = TSHQLSTMEvalParametersProto(
        input_width=1,
        label_width=1,
        hidden_size=17,
        num_qubits=8,
        depth=7,
        n_qlayers=3,
        dropout_coef=0.24,
        time_column="Time",
    ),
    inputs=TSEvalInputsProto(
        model=ModelStorageInfoProto(storage_id="MODEL_BUCKET_STORAGE_ID"),
        data=DatasetStorageInfoProto(storage_id="DATA_BUCKET_STORAGE_ID")
    )  
), preserving_proto_field_name=True)

# Evaluate the model

In [18]:
run = ExperimentRun.create(
    client=client,
    algorithm=AlgorithmProto.TS_HQLSTM_EVAL,
    experiment_id=exp_sample.id,
    compute=HardwareProto.SMALL,
    parameters=params_eval
)
print(run.data)

id: "5e799d86-6ee9-4c90-99bd-5ffb2a3d368a"
experiment_id: "811fd59b-2b51-42ea-b573-b5aebb0d5d7a"
sequential_id: 27
status: QUEUED
algorithm: TS_HQLSTM_EVAL
hardware: SMALL
metadata: "{\"parameters\":{\"input_width\":1,\"label_width\":1,\"hidden_size\":17,\"num_qubits\":8,\"depth\":7,\"dropout_coef\":0.24,\"nqlayers\":3},\"inputs\":{\"model\":{\"storage_id\":\"MODEL_BUCKET_STORAGE_ID\"},\"data\":{\"storage_id\":\"DATA_BUCKET_STORAGE_ID\"}}}"
result {
}
created_by: "35dc8cba-b06d-4f15-b9bc-588f608789de"
created_at {
  seconds: 1705664558
  nanos: 249253227
}
